# Ottimizzazione dei modelli Open AI

Questo notebook si basa sulle indicazioni attuali fornite nella documentazione [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) di Open AI.

L’ottimizzazione migliora le prestazioni dei modelli di base per la tua applicazione, riaddestrandoli con dati e contesti aggiuntivi rilevanti per quello specifico caso d’uso o scenario. Tieni presente che tecniche di prompt engineering come _few shot learning_ e _retrieval augmented generation_ ti permettono di arricchire il prompt predefinito con dati pertinenti per migliorare la qualità. Tuttavia, questi approcci sono limitati dalla dimensione massima della finestra di token del modello di base scelto.

Con l’ottimizzazione, stiamo di fatto riaddestrando il modello stesso con i dati necessari (il che ci consente di usare molti più esempi rispetto a quanti ne entrerebbero nella finestra massima di token) e distribuiamo una versione _personalizzata_ del modello che non ha più bisogno di ricevere esempi durante l’inferenza. Questo non solo migliora l’efficacia della progettazione del prompt (abbiamo più flessibilità nell’uso della finestra di token per altri scopi), ma può anche ridurre i costi (diminuendo il numero di token da inviare al modello in fase di inferenza).

L’ottimizzazione prevede 4 passaggi:
1. Prepara i dati di addestramento e caricali.
1. Avvia il job di addestramento per ottenere un modello ottimizzato.
1. Valuta il modello ottimizzato e ripeti per migliorarne la qualità.
1. Distribuisci il modello ottimizzato per l’inferenza quando sei soddisfatto.

Tieni presente che non tutti i modelli di base supportano l’ottimizzazione: [consulta la documentazione OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) per le informazioni più aggiornate. Puoi anche ottimizzare un modello che è già stato ottimizzato in precedenza. In questo tutorial useremo `gpt-35-turbo` come modello di base di riferimento per l’ottimizzazione.

---


### Passaggio 1.1: Prepara il tuo dataset

Costruiamo un chatbot che ti aiuti a comprendere la tavola periodica degli elementi rispondendo alle domande su un elemento con una filastrocca in rima. In _questo_ semplice tutorial, ci limiteremo a creare un dataset per addestrare il modello con alcuni esempi di risposte che mostrano il formato atteso dei dati. In un caso d’uso reale, dovresti creare un dataset con molti più esempi. Potresti anche utilizzare un dataset open source (per il tuo dominio applicativo) se esiste, e riformattarlo per l’uso nel fine-tuning.

Poiché ci concentriamo su `gpt-35-turbo` e vogliamo una risposta a turno singolo (chat completion), possiamo creare esempi usando [questo formato suggerito](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) che rispecchia i requisiti di completamento chat di OpenAI. Se ti aspetti contenuti conversazionali a più turni, dovresti usare il [formato per esempi multi-turno](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) che include un parametro `weight` per indicare quali messaggi devono essere usati (o meno) nel processo di fine-tuning.

Per il nostro tutorial useremo il formato più semplice a turno singolo. I dati sono nel [formato jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) con 1 record per riga, ciascuno rappresentato come un oggetto in formato JSON. Il frammento qui sotto mostra 2 record come esempio - vedi [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) per il set completo di esempio (10 esempi) che useremo per il nostro tutorial di fine-tuning. **Nota:** Ogni record _deve_ essere definito su una singola riga (non suddiviso su più righe come avviene di solito in un file JSON formattato)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

In un caso d’uso reale avrai bisogno di un set di esempi molto più ampio per ottenere buoni risultati: il compromesso sarà tra la qualità delle risposte e il tempo/costo del fine-tuning. Noi usiamo un set piccolo così da completare rapidamente il fine-tuning e illustrare il processo. Consulta [questo esempio OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) per un tutorial di fine-tuning più complesso.


### Passaggio 1.2 Carica il tuo dataset

Carica i dati utilizzando le API Files [come descritto qui](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Nota che, per eseguire questo codice, devi aver completato prima i seguenti passaggi:
 - Installato il pacchetto Python `openai` (assicurati di utilizzare una versione >=0.28.0 per avere le funzionalità più recenti)
 - Impostato la variabile d'ambiente `OPENAI_API_KEY` con la tua chiave API di OpenAI
Per saperne di più, consulta la [guida alla configurazione](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) fornita per il corso.

Ora, esegui il codice per creare un file da caricare a partire dal tuo file JSONL locale.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Passaggio 2.1: Crea il job di fine-tuning con l’SDK



In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Passaggio 2.2: Controlla lo stato del job

Ecco alcune cose che puoi fare con l’API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Elenca gli ultimi n job di fine-tuning
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Ottieni i dettagli di uno specifico job di fine-tuning
- `client.fine_tuning.jobs.cancel(<job_id>)` - Annulla un job di fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Elenca fino a n eventi del job
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Il primo passo del processo è _validare il file di training_ per assicurarsi che i dati siano nel formato corretto.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Passaggio 2.3: Traccia gli eventi per monitorare i progressi


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Passaggio 2.4: Visualizza lo stato nella Dashboard di OpenAI


Puoi anche visualizzare lo stato visitando il sito di OpenAI ed esplorando la sezione _Fine-tuning_ della piattaforma. Qui potrai vedere lo stato del job attuale e anche tenere traccia della cronologia delle esecuzioni precedenti. In questo screenshot, puoi notare che l’esecuzione precedente non è andata a buon fine, mentre la seconda è riuscita. Per darti un’idea, questo è successo perché la prima esecuzione utilizzava un file JSON con record formattati in modo errato – una volta corretto il problema, la seconda esecuzione è stata completata con successo e il modello è diventato disponibile per l’uso.

![Stato del job di fine-tuning](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.it.png)


Puoi anche visualizzare i messaggi di stato e le metriche scorrendo più in basso nella dashboard visiva come mostrato:

| Messaggi | Metriche |
|:---|:---|
| ![Messaggi](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.it.png) |  ![Metriche](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.it.png)|


### Passaggio 3.1: Recupera l'ID e testa il modello ottimizzato nel codice


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Passaggio 3.2: Carica e testa il modello ottimizzato nel Playground

Ora puoi testare il modello ottimizzato in due modi. Innanzitutto, puoi visitare il Playground e usare il menu a tendina dei Modelli per selezionare il tuo nuovo modello ottimizzato tra le opzioni disponibili. L’altra opzione è utilizzare la voce "Playground" mostrata nel pannello di Fine-tuning (vedi screenshot sopra), che avvia la seguente vista _comparativa_ mostrando fianco a fianco la versione base e quella ottimizzata del modello per una valutazione rapida.

![Stato del job di fine-tuning](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.it.png)

Basta inserire il contesto di sistema usato nei tuoi dati di addestramento e fornire la domanda di test. Noterai che entrambi i lati vengono aggiornati con lo stesso contesto e la stessa domanda. Avvia il confronto e vedrai la differenza tra le risposte generate. _Nota come il modello ottimizzato restituisce la risposta nel formato che hai fornito nei tuoi esempi, mentre il modello base segue semplicemente il prompt di sistema_.

![Stato del job di fine-tuning](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.it.png)

Noterai anche che il confronto mostra il conteggio dei token per ciascun modello e il tempo impiegato per l’inferenza. **Questo esempio specifico è molto semplice e serve solo a mostrare il processo, ma non riflette un vero dataset o scenario reale**. Potresti notare che entrambi gli esempi mostrano lo stesso numero di token (il contesto di sistema e il prompt utente sono identici), ma il modello ottimizzato impiega più tempo per l’inferenza (modello personalizzato).

Nella pratica reale, non userai un esempio così semplice, ma ottimizzerai il modello su dati reali (ad esempio, un catalogo prodotti per l’assistenza clienti) dove la qualità della risposta sarà molto più evidente. In _quel_ contesto, ottenere una qualità di risposta equivalente con il modello base richiederà una maggiore personalizzazione del prompt, aumentando così l’uso di token e potenzialmente anche il tempo di elaborazione per l’inferenza. _Per provare, dai un’occhiata agli esempi di fine-tuning nell’OpenAI Cookbook per iniziare._



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Pur impegnandoci per garantire l’accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa deve essere considerato la fonte autorevole. Per informazioni di carattere critico, si raccomanda una traduzione professionale effettuata da un essere umano. Non siamo responsabili per eventuali malintesi o interpretazioni errate derivanti dall’uso di questa traduzione.
